In [10]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.linear_model import RidgeClassifierCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

# -----------------------------
# XGBoost (ทำให้ไม่ล้มแม้ยังไม่ติดตั้ง)
# -----------------------------
try:
    from xgboost import XGBClassifier  # pip install xgboost
    HAS_XGB = True
except Exception:
    HAS_XGB = False

# =============================
# PATH / Utils / Labels / Flags
# =============================
file_path = r"C:\Users\piriy\Desktop\dataset.xlsx"
out_path  = r"C:\Users\piriy\Desktop\dataset_with_final_weights_groupaware_1.xlsx"

RANDOM_SEED = 20250913
rng = np.random.default_rng(RANDOM_SEED)

# --- Production flags & Policies ---
ENABLE_NESTED_CV_CHECK   = True     # รายงาน Nested-CV ของ threshold tuning (ควรเชื่ออันนี้มากกว่า)
MIN_GAP                  = 0.08     # ลดกันช่วงขั้นต่ำเพื่อไม่ให้ threshold แข็งเกินไป
COARSE_STEP              = 0.15     # coarse grid (ใช้เฉพาะตอนค้นหา ไม่ใช้กับ policy-lock)
POLICY_LOCK_QUANTILES    = True     # ใช้ q20/40/60/80 (หรือ target-quantile) จากฝั่ง damage โดยตรง
USE_TARGET_PREVALENCE    = True     # ใช้ "เป้าสัดส่วน" ต่อชั้นเพื่อคุมภาระงาน (ถ้า False จะใช้ q20/40/60/80)
OPTIMIZE_FOR             = "f1"     # "f1" หรือ "cost" (ถ้า "cost" จะใช้ COST_MATRIX ด้านล่าง)

# เป้าสัดส่วน (เฉพาะฝั่ง "ที่มีความเสียหาย") รวม 100
# [น้อยมาก, น้อย, ปานกลาง, รุนแรง, วิกฤต]
# หมายเหตุ: top2 = 8%+4% = 12%
TARGET_PREVALENCE = np.array([46, 30, 12, 8, 4], dtype=float)

# Cost matrix สำหรับโหมด "cost" (ค่าตัวอย่าง หลงทางหนักเมื่อพลาดชั้นบน)
# แกนแถว = true, แกนคอลัมน์ = pred ; ลำดับคลาส = labels_6_order ด้านล่าง
COST_MATRIX = np.array([
# pred:   0    1    2    3    4    5
          [0,   1,   2,   3,   4,   5],  # true 0 (ไม่มีความเสียหาย)
          [1,   0,   1,   2,   3,   4],  # true 1
          [2,   1,   0,   1,   2,   3],  # true 2
          [3,   2,   1,   0,   2,   3],  # true 3
          [5,   4,   3,   2,   0,   2],  # true 4
          [8,   6,   5,   4,   2,   0],  # true 5 (วิกฤต)
], dtype=float)

labels_5 = ["น้อยมาก", "น้อย", "ปานกลาง", "รุนแรง", "วิกฤต"]
labels_6_order = ["ไม่มีความเสียหาย"] + labels_5
code_map = {"ไม่มีความเสียหาย": 0, "น้อยมาก": 1, "น้อย": 2, "ปานกลาง": 3, "รุนแรง": 4, "วิกฤต": 5}

# ---- sanitize thresholds (มีกันช่วงบาง) & ค่าเริ่มต้น ----
def sanitize_thresholds(th, eps=1e-6, min_gap=MIN_GAP):
    th = np.asarray(th, dtype=float).tolist()
    th = [min(max(0.0, t), 1.0) for t in th]  # clip [0,1]
    th = sorted(th)

    # เดินหน้า: บังคับเพิ่มอย่างน้อย min_gap
    for i in range(1, len(th)):
        need = th[i-1] + max(eps, min_gap)
        if th[i] < need:
            th[i] = min(1.0, need)

    # ไล่กลับ: คง min_gap จากขวาไปซ้าย
    for i in range(len(th)-2, -1, -1):
        need = th[i+1] - max(eps, min_gap)
        if th[i] > need:
            th[i] = max(0.0, need)
    return th

THRESHOLDS = sanitize_thresholds([0.20, 0.40, 0.60, 0.80])  # จะถูกอัพเดตภายหลัง

# ---- nudge_top2: จูน t3/t4 ให้สัดส่วน top2 ใกล้เป้าหมาย ----
def nudge_top2(th, X01, mask_dmg, target_pct=12.0, tol=0.2, step=0.002, max_iter=800):
    """
    ขยับ t3/t4 เล็กน้อยเพื่อดันสัดส่วน top2 (รุนแรง+วิกฤต) ให้ใกล้ target_pct (บนฝั่งที่มีความเสียหาย)
    - ขึ้น t3/t4  -> ลด top2
    - ลง t3/t4    -> เพิ่ม top2
    """
    th = np.array(sanitize_thresholds(th), dtype=float)
    X01 = np.asarray(X01, dtype=float)
    mb = np.asarray(mask_dmg, dtype=bool)
    Xd = X01[mb]
    if Xd.size == 0:
        return th.tolist()

    def top2_pct(th_):
        lab = bin_by_thresholds(Xd, thresholds=th_, labels=labels_5)  # pandas.Categorical
        arr = np.asarray(lab, dtype=object)                            # -> ndarray (object)
        p = np.isin(arr, ["รุนแรง", "วิกฤต"]).mean() * 100.0         # ใช้ np.isin แทน .isin
        return float(p)

    cur = top2_pct(th)
    for _ in range(max_iter):
        if abs(cur - target_pct) <= tol:
            break
        if cur > target_pct:   # มากไป → ดันเกณฑ์ขึ้น
            th[3] = min(1.0, th[3] + step)                   # ขยับ t4 ก่อน
            th[2] = min(th[3] - MIN_GAP, th[2] + step/2)     # รักษา MIN_GAP
        else:                  # น้อยไป → ผ่อนเกณฑ์ลง
            th[3] = max(th[2] + MIN_GAP, th[3] - step)
            th[2] = max(th[1] + MIN_GAP, th[2] - step/2)
        th = np.array(sanitize_thresholds(th), dtype=float)
        cur = top2_pct(th)
    return th.tolist()


def normalize_weights(w: np.ndarray) -> np.ndarray:
    w = np.asarray(w, dtype=float)
    w[w < 0] = 0.0
    s = w.sum()
    return (w / s) if s > 0 else np.ones_like(w) / len(w)

def new_splitter(X, y, groups):
    unique_groups = getattr(groups, "nunique", lambda: pd.Series(groups).nunique())()
    if unique_groups >= 5:
        return GroupKFold(n_splits=5).split(X, y, groups)
    else:
        print(f"⚠️ กลุ่มมีเพียง {unique_groups} < 5 → ใช้ StratifiedKFold(5)")
        return StratifiedKFold(n_splits=5, shuffle=True, random_state=42).split(X, y)

# -------- robust CV helper (ใช้ LogisticRegression เป็น meta-evaluator) --------
def macro_f1_cv_robust(X: np.ndarray, y: np.ndarray, groups: pd.Series) -> float:
    splitter = list(new_splitter(X, y, groups))
    scores = []
    for tr_idx, va_idx in splitter:
        X_tr, X_va = X[tr_idx], X[va_idx]
        y_tr, y_va = y[tr_idx], y[va_idx]

        if np.unique(y_va).shape[0] < 2:
            scores.append(0.0)
            continue

        clf = LogisticRegression(
            max_iter=2000,
            class_weight="balanced",
            random_state=RANDOM_SEED,
            solver="lbfgs",
        )
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_va)
        f1 = f1_score(y_va, y_pred, average="macro", zero_division=0)
        scores.append(float(f1))
    return float(np.mean(scores)) if len(scores) > 0 else 0.0

# -------- ใช้ robust CV ประเมิน weights --------
def evaluate_weights(w: np.ndarray, X, y, groups) -> float:
    w = np.asarray(w, dtype=float)
    if w.shape[0] != X.shape[1]:
        raise ValueError(f"weight length={len(w)} != n_features={X.shape[1]}")
    dmg = X @ w
    mn, mx = np.nanmin(dmg), np.nanmax(dmg)
    dmg_norm = (dmg - mn) / (mx - mn) if mx > mn else np.zeros_like(dmg)
    X_feat = dmg_norm.reshape(-1, 1)
    return macro_f1_cv_robust(X_feat, np.asarray(y, dtype=int), groups)

def bin_by_thresholds(arr01: np.ndarray, thresholds=THRESHOLDS, labels=labels_5) -> pd.Categorical:
    """
    ติดป้าย 5 ระดับบนสเกล [0,1]: [0,t1) [t1,t2) [t2,t3) [t3,t4) [t4,1]
    """
    thresholds = sanitize_thresholds(thresholds)
    arr = np.nan_to_num(np.asarray(arr01, dtype=float), nan=0.0, posinf=1.0, neginf=0.0)
    bins = np.digitize(arr, thresholds, right=False)  # 0..len(thresholds)
    bins = np.clip(bins, 0, len(labels)-1)
    lab = np.array(labels, dtype=object)[bins]
    return pd.Categorical(lab, categories=labels, ordered=True)

# =============================
# Load + Check
# =============================
df = pd.read_excel(file_path)

base_cols = ["UP_DMG", "WALL_DMG", "FLOOR_DMG", "PILLAR_DMG", "STRUCTURE_"]
missing = [c for c in base_cols if c not in df.columns]
if missing:
    raise ValueError(f"ไม่พบคอลัมน์ที่จำเป็น: {missing}")

df[base_cols] = df[base_cols].apply(pd.to_numeric, errors="coerce").fillna(0)

# =============================
# แยกแถวมี/ไม่มีความเสียหาย
# =============================
mask_has_damage = df[base_cols].sum(axis=1) > 0
print(f"✅ มีความเสียหาย: {int(mask_has_damage.sum())} แถว | ไม่มีความเสียหาย: {int((~mask_has_damage).sum())} แถว")

# =============================
# Normalize: q95 จากฝั่งที่มีความเสียหาย แล้วใช้ทั้งก้อน
# =============================
norm_q95 = {}
df_damage_for_norm = df.loc[mask_has_damage].copy()
for c in base_cols:
    q95 = df_damage_for_norm[c].quantile(0.95)
    q95 = 1.0 if pd.isna(q95) or q95 == 0 else float(q95)
    norm_q95[c] = q95
    df[f"{c}_NORM"] = df[c].clip(upper=q95) / q95

norm_cols = [f"{c}_NORM" for c in base_cols]
df[norm_cols] = df[norm_cols].fillna(0.0)

# =============================
# Target 6 คลาส (seed label เริ่มต้นด้วย thresholds เริ่มต้น)
# =============================
tmp_damage_score = df.loc[mask_has_damage, norm_cols].mean(axis=1).to_numpy()
dmg_levels_5_fixed = bin_by_thresholds(tmp_damage_score, thresholds=THRESHOLDS, labels=labels_5)

y6 = pd.Series(index=df.index, dtype=object)
y6.loc[~mask_has_damage] = "ไม่มีความเสียหาย"
y6.loc[mask_has_damage]  = dmg_levels_5_fixed.astype(str)

le6 = LabelEncoder()
le6.fit(labels_6_order)  # ล็อกลำดับคลาส (ไทย)
y6_enc = le6.transform(y6.astype(str))

# =============================
# กลุ่มสำหรับ GroupKFold
# =============================
if "DISTRICT" in df.columns and "SUB_DISTRI" in df.columns:
    groups_all = df["DISTRICT"].astype(str) + " | " + df["SUB_DISTRI"].astype(str)
else:
    print("⚠️ ไม่พบ DISTRICT หรือ SUB_DISTRI → จะใช้ StratifiedKFold")
    groups_all = pd.Series(["__all__"] * len(df))

# =============================
# ฟีเจอร์ + Weights จากหลายวิธี (6 คลาสทั้งก้อน)
# =============================
X_all = df[norm_cols].to_numpy()

# 1) Expert prior
expert_weights = normalize_weights(np.array([0.15, 0.10, 0.10, 0.25, 0.40]))

# 2) Mutual Information
mi6 = mutual_info_classif(X_all, y6_enc, random_state=42, discrete_features=False)
mi6_weights = normalize_weights(mi6)

# 3) Ridge (สรุปค่าสัมบูรณ์ข้ามชั้น)
ridge6 = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
ridge6.fit(X_all, y6_enc)
ridge6_coef = ridge6.coef_
ridge6_feat = np.abs(ridge6_coef) if ridge6_coef.ndim == 1 else np.abs(ridge6_coef).sum(axis=0)
ridge6_weights = normalize_weights(ridge6_feat)

# 4) RandomForest
rf6 = RandomForestClassifier(n_estimators=300, random_state=42, class_weight="balanced", n_jobs=-1)
rf6.fit(X_all, y6_enc)
rf6_weights = normalize_weights(rf6.feature_importances_)

# 5) XGBoost (ถ้าไม่มี ให้ข้าม)
if HAS_XGB:
    xgb6 = XGBClassifier(
        n_estimators=400, learning_rate=0.05, max_depth=3,
        subsample=0.8, colsample_bytree=0.8,
        eval_metric="mlogloss", random_state=42,
        n_jobs=-1,
    )
    xgb6.fit(X_all, y6_enc)
    xgb6_weights = normalize_weights(xgb6.feature_importances_)
else:
    xgb6_weights = None

weights_dict6 = {
    "ผู้เชี่ยวชาญ": expert_weights,
    "MI":     mi6_weights,
    "Ridge":  ridge6_weights,
    "RF":     rf6_weights,
}
if xgb6_weights is not None:
    weights_dict6["XGB"] = xgb6_weights

# =============================
# Evaluate weights (6-class, group-aware) → Best method
# =============================
scores6 = {name: evaluate_weights(w, X_all, y6_enc, groups_all) for name, w in weights_dict6.items()}
cv_df6 = pd.DataFrame.from_dict(scores6, orient="index", columns=["Macro-F1 (6 คลาส)"]).sort_values("Macro-F1 (6 คลาส)", ascending=False)
print("📊 คะแนน CV Macro-F1 (6 คลาส, group-aware):")
print(cv_df6)

best_method6 = cv_df6.index[0]
final_weights6 = weights_dict6[best_method6]
print(f"\n✅ วิธีที่ดีที่สุด (6 คลาส): {best_method6}")

# =============================
# คำนวณสกอร์รวมด้วย weights ที่ดีที่สุด (ไว้ใช้ 'จูน thresholds')
# =============================
TOTAL_all = X_all @ final_weights6
mn_all, mx_all = np.nanmin(TOTAL_all), np.nanmax(TOTAL_all)
total_all_norm = (TOTAL_all - mn_all) / (mx_all - mn_all) if mx_all > mn_all else np.zeros_like(TOTAL_all)

# --------- ฟังก์ชันวัดผลตาม thresholds ----------
def macro_f1_for_thresholds_full(th):
    th = sanitize_thresholds(th)
    y6_th = pd.Series(index=df.index, dtype=object)
    y6_th.loc[~mask_has_damage] = "ไม่มีความเสียหาย"
    y6_th.loc[mask_has_damage] = bin_by_thresholds(
        total_all_norm[mask_has_damage], thresholds=th, labels=labels_5
    ).astype(str)
    le = LabelEncoder().fit(labels_6_order)
    y_enc = le.transform(y6_th.astype(str))
    X_feat = total_all_norm.reshape(-1, 1)
    return macro_f1_cv_robust(X_feat, y_enc, groups_all)

def expected_cost_for_thresholds(th):
    th = sanitize_thresholds(th)
    y_pred = pd.Series("ไม่มีความเสียหาย", index=df.index, dtype=object)
    y_pred.loc[mask_has_damage] = bin_by_thresholds(
        total_all_norm[mask_has_damage], thresholds=th, labels=labels_5
    ).astype(str)
    enc = LabelEncoder().fit(labels_6_order)
    yp = enc.transform(y_pred.astype(str))
    yt = enc.transform(y6.astype(str))
    return COST_MATRIX[yt, yp].mean()

def coarse_grid_candidates(step=COARSE_STEP):
    grid = np.arange(step, 1.0, step)
    cand = []
    for t1 in grid:
        for t2 in grid:
            if t2 <= t1: continue
            for t3 in grid:
                if t3 <= t2: continue
                for t4 in grid:
                    if t4 <= t3: continue
                    cand.append([t1, t2, t3, t4])
    return cand

def local_refine(th_best, n_iter=300, sigma=0.04, objective="f1"):
    best = np.array(th_best, dtype=float)
    if objective == "f1":
        best_score = macro_f1_for_thresholds_full(best)
        better = lambda s, b: s > b
    else:
        best_score = expected_cost_for_thresholds(best)
        better = lambda s, b: s < b  # minimize cost

    for _ in range(n_iter):
        prop = best + rng.normal(0, sigma, size=4)
        prop = sanitize_thresholds(prop)
        score = macro_f1_for_thresholds_full(prop) if objective == "f1" else expected_cost_for_thresholds(prop)
        if better(score, best_score):
            best, best_score = np.array(prop), score
    return best.tolist(), best_score

# ============ 1) หา "จุดเริ่ม" ของ thresholds ============
if mask_has_damage.any():
    if POLICY_LOCK_QUANTILES and USE_TARGET_PREVALENCE:
        q = np.cumsum(TARGET_PREVALENCE[:-1] / TARGET_PREVALENCE.sum())
        start_th = sanitize_thresholds(np.quantile(total_all_norm[mask_has_damage], q))
        # จูนละเอียดให้ top2 ≈ 12% (บนฝั่งที่มีความเสียหาย)
        start_th = nudge_top2(start_th, total_all_norm, mask_has_damage,
                              target_pct=12.0, tol=0.2)
    else:
        q = np.quantile(total_all_norm[mask_has_damage], [0.2, 0.4, 0.6, 0.8])
        start_th = sanitize_thresholds(q)
else:
    start_th = THRESHOLDS

# ============ 2) เลือกเกณฑ์ตาม Policy / หรือปรับจูน ============
if POLICY_LOCK_QUANTILES:
    THRESHOLDS = start_th
    if OPTIMIZE_FOR == "cost":
        score_txt = expected_cost_for_thresholds(THRESHOLDS)
        print(f"🔒 เกณฑ์(Thresholds) policy-lock: {THRESHOLDS} | Expected Cost = {score_txt:.4f}")
    else:
        score_txt = macro_f1_for_thresholds_full(THRESHOLDS)
        print(f"🔒 เกณฑ์(Thresholds) policy-lock: {THRESHOLDS} | CV Macro-F1(6 คลาส) = {score_txt:.4f}")
else:
    cands = [[*start_th]] + coarse_grid_candidates(step=COARSE_STEP)
    if OPTIMIZE_FOR == "cost":
        best_th, best_score = None, np.inf
        for th in cands:
            s = expected_cost_for_thresholds(th)
            if s < best_score:
                best_th, best_score = th, s
        best_th, best_score = local_refine(best_th, n_iter=300, sigma=0.04, objective="cost")
        THRESHOLDS = sanitize_thresholds(best_th)
        print(f"🎯 เกณฑ์(Thresholds) ที่ดีที่สุด (min Expected Cost, step={COARSE_STEP}, min_gap={MIN_GAP:.02f}): {THRESHOLDS} | Cost = {best_score:.4f}")
    else:
        best_th, best_score = None, -1.0
        for th in cands:
            s = macro_f1_for_thresholds_full(th)
            if s > best_score:
                best_th, best_score = th, s
        best_th, best_score = local_refine(best_th, n_iter=300, sigma=0.04, objective="f1")
        THRESHOLDS = sanitize_thresholds(best_th)
        print(f"🎯 เกณฑ์(Thresholds) ที่ดีที่สุด (ปรับจูน F1, step={COARSE_STEP}, min_gap={MIN_GAP:.02f}): {THRESHOLDS} | CV Macro-F1 = {best_score:.4f}")

# ============ 3) Nested-CV (เชื่ออันนี้มากกว่า) ============
nested_cv_score = None
if ENABLE_NESTED_CV_CHECK:
    splitter_outer = list(new_splitter(total_all_norm.reshape(-1,1), le6.transform(y6.astype(str)), groups_all))
    scores_nested = []
    groups_series = groups_all if isinstance(groups_all, pd.Series) else pd.Series(groups_all)

    for tr_idx, va_idx in splitter_outer:
        X_tr_norm = total_all_norm[tr_idx]
        mask_tr = mask_has_damage.values[tr_idx]
        groups_tr = groups_series.iloc[tr_idx]

        # ตั้งต้น threshold บน train
        if mask_tr.any():
            if POLICY_LOCK_QUANTILES and USE_TARGET_PREVALENCE:
                qt = np.cumsum(TARGET_PREVALENCE[:-1] / TARGET_PREVALENCE.sum())
                start_th0 = sanitize_thresholds(np.quantile(X_tr_norm[mask_tr], qt))
            else:
                start_th0 = sanitize_thresholds(np.quantile(X_tr_norm[mask_tr], [0.2,0.4,0.6,0.8]))
        else:
            start_th0 = THRESHOLDS

        if POLICY_LOCK_QUANTILES:
            best_th0 = start_th0
        else:
            def score_th_on_train(th):
                y_tr_series = pd.Series("ไม่มีความเสียหาย", index=groups_tr.index, dtype=object)
                y_tr_series.loc[mask_tr] = bin_by_thresholds(
                    X_tr_norm[mask_tr], thresholds=th, labels=labels_5
                ).astype(str)
                le_tmp = LabelEncoder().fit(labels_6_order)
                y_tr_enc = le_tmp.transform(y_tr_series.astype(str))
                # ในโหมด cost ให้กลับหัวเป็น -cost เพื่อใช้ logic เดิม (maximize)
                if OPTIMIZE_FOR == "cost":
                    yp = le_tmp.transform(y_tr_series.astype(str))
                    yt = le_tmp.transform(y6.iloc[tr_idx].astype(str))
                    return -COST_MATRIX[yt, yp].mean()
                return macro_f1_cv_robust(X_tr_norm.reshape(-1,1), y_tr_enc, groups_tr)

            cands0 = [[*start_th0]] + coarse_grid_candidates(step=COARSE_STEP)
            best_th0, best_sc0 = None, -np.inf
            for th in cands0:
                sc = score_th_on_train(th)
                if sc > best_sc0:
                    best_th0, best_sc0 = th, sc
            # refine
            obj = "cost" if OPTIMIZE_FOR == "cost" else "f1"
            best_th0, _ = local_refine(best_th0, n_iter=150, sigma=0.03, objective=obj)

        # ประเมินบน valid fold
        X_va_norm = total_all_norm[va_idx]
        mask_va = mask_has_damage.values[va_idx]
        groups_va = groups_series.iloc[va_idx]

        y_va_pred = pd.Series("ไม่มีความเสียหาย", index=groups_va.index, dtype=object)
        y_va_pred.loc[mask_va] = bin_by_thresholds(
            X_va_norm[mask_va], thresholds=best_th0, labels=labels_5
        ).astype(str)

        y_va_true = y6.iloc[va_idx]
        le_tmp = LabelEncoder().fit(labels_6_order)

        if OPTIMIZE_FOR == "cost":
            yt = le_tmp.transform(y_va_true.astype(str))
            yp = le_tmp.transform(y_va_pred.astype(str))
            scores_nested.append(float(-COST_MATRIX[yt, yp].mean()))  # เก็บเป็นค่าที่มากดีกว่า
        else:
            f1_va = f1_score(
                le_tmp.transform(y_va_true.astype(str)),
                le_tmp.transform(y_va_pred.astype(str)),
                average="macro", zero_division=0
            )
            scores_nested.append(float(f1_va))

    nested_cv_score = float(np.mean(scores_nested)) if scores_nested else None
    mode_txt = ("policy-lock + target-quantile" if (POLICY_LOCK_QUANTILES and USE_TARGET_PREVALENCE)
                else ("policy-lock q20/40/60/80" if POLICY_LOCK_QUANTILES
                      else f"tuned (step={COARSE_STEP})"))
    metric_txt = "Macro-F1" if OPTIMIZE_FOR == "f1" else "-ExpectedCost (ยิ่งมากยิ่งดี)"
    print(f"🧪 Nested-CV {metric_txt} (threshold mode: {mode_txt}, min_gap={MIN_GAP:.02f}): {nested_cv_score:.4f}")

# =============================
# Bootstrap Stability (เช็คความเสถียรของ thresholds/สัดส่วน)
# =============================
def bootstrap_threshold_stability(X01, mask_dmg, groups, thresholds, B=200, seed=2025):
    rng = np.random.default_rng(seed)
    groups = pd.Series(groups)
    uniq = groups.unique()
    th_list = []
    counts = []
    for _ in range(B):
        sel_g = rng.choice(uniq, size=len(uniq), replace=True)
        idx = groups.isin(sel_g).values
        Xb = X01[idx]; mb = mask_dmg.values[idx]
        if mb.any():
            qb = np.quantile(Xb[mb], [0.2,0.4,0.6,0.8]) if not USE_TARGET_PREVALENCE else \
                 np.quantile(Xb[mb], np.cumsum(TARGET_PREVALENCE[:-1] / TARGET_PREVALENCE.sum()))
            qb = sanitize_thresholds(qb)
        else:
            qb = sanitize_thresholds(thresholds)
        th_list.append(qb)
        # class counts under this qb
        lab = pd.Series("ไม่มีความเสียหาย", index=np.arange(Xb.shape[0]), dtype=object)
        lab.loc[mb] = bin_by_thresholds(Xb[mb], thresholds=qb, labels=labels_5).astype(str)
        counts.append(lab.value_counts().reindex(["ไม่มีความเสียหาย"]+labels_5, fill_value=0).to_dict())
    th_arr = np.array(th_list)
    return th_arr.mean(0), th_arr.std(0), counts

th_mean, th_std, boot_counts = bootstrap_threshold_stability(
    total_all_norm, mask_has_damage, groups_all, THRESHOLDS, B=200, seed=2025
)
print(f"🔁 Bootstrap thresholds mean: {np.round(th_mean, 4).tolist()} | std: {np.round(th_std, 4).tolist()}")

# =============================
# Apply weights → Final Score & Class (ใช้ thresholds ที่สรุปได้)
# =============================
df["TOTAL_DMG_FINAL"] = TOTAL_all
df["TOTAL_DMG_FINAL_NORM"] = np.nan_to_num(total_all_norm, nan=0.0, posinf=1.0, neginf=0.0)

df["DMG_LEVEL_FINAL"] = None
df.loc[~mask_has_damage, "DMG_LEVEL_FINAL"] = "ไม่มีความเสียหาย"
df.loc[mask_has_damage, "DMG_LEVEL_FINAL"] = bin_by_thresholds(
    df.loc[mask_has_damage, "TOTAL_DMG_FINAL_NORM"].to_numpy(),
    thresholds=THRESHOLDS, labels=labels_5
).astype(str)

df["DMG_CODE"] = df["DMG_LEVEL_FINAL"].map(code_map).astype("Int64")

print("\nการกระจายระดับความเสียหาย (หลังตั้งเกณฑ์):")
dist_final = df["DMG_LEVEL_FINAL"].value_counts(dropna=False)
print(dist_final)

# =============================
# สร้างรายงานประสิทธิภาพรวม (อิง label seed y6)
# =============================
le_final = LabelEncoder().fit(labels_6_order)
y_true_enc = le_final.transform(y6.astype(str))

pred_labels = pd.Series("ไม่มีความเสียหาย", index=df.index, dtype=object)
pred_labels.loc[mask_has_damage] = bin_by_thresholds(
    df.loc[mask_has_damage, "TOTAL_DMG_FINAL_NORM"].to_numpy(),
    thresholds=THRESHOLDS, labels=labels_5
).astype(str)
y_pred_enc = le_final.transform(pred_labels.astype(str))

cls_report = classification_report(
    y_true_enc, y_pred_enc, target_names=labels_6_order, zero_division=0, output_dict=True
)
report_df = pd.DataFrame(cls_report).transpose()
# เปลี่ยนชื่อคอลัมน์รายงานเป็นไทย
report_df = report_df.rename(columns={
    "precision": "ความแม่นยำ (Precision)",
    "recall": "การครอบคลุม (Recall)",
    "f1-score": "F1-Score",
    "support": "จำนวนตัวอย่าง",
})

cm = confusion_matrix(
    y_true_enc, y_pred_enc,
    labels=list(range(len(labels_6_order)))
)
cm_df = pd.DataFrame(cm, index=labels_6_order, columns=labels_6_order)

# =============================
# ตารางเสริม: น้ำหนัก, พารามิเตอร์, เมตา, Stability สรุป
# =============================
weights_table = pd.DataFrame({"ฟีเจอร์": base_cols, "น้ำหนัก": final_weights6})
norm_params = pd.DataFrame({
    "ฟีเจอร์": list(norm_q95.keys()),
    "ค่า q95 ที่ใช้": list(norm_q95.values()),
    "หมายเหตุ": ["ตัดบนที่ q95 (จากฝั่งที่มีความเสียหาย) แล้วหารด้วย q95"] * len(norm_q95)
})

# แพ็ครายละเอียด stability ของสัดส่วนแต่ละชั้นจาก bootstrap (เฉลี่ย)
def summarize_boot_counts(boot_counts_list):
    keys = ["ไม่มีความเสียหาย"] + labels_5
    arr = np.array([[d[k] for k in keys] for d in boot_counts_list], dtype=float)
    mean = arr.mean(0); std = arr.std(0)
    total = mean.sum()
    pct = mean / total * 100.0
    return pd.DataFrame({
        "ชั้น": keys,
        "จำนวนเฉลี่ย (bootstrap)": np.round(mean, 2),
        "ส่วนเบี่ยงเบนมาตรฐาน": np.round(std, 2),
        "สัดส่วนเฉลี่ย (%)": np.round(pct, 2),
    })

stability_df = summarize_boot_counts(boot_counts)
th_stability_df = pd.DataFrame({
    "Threshold": ["t1","t2","t3","t4"],
    "ค่าเฉลี่ย (bootstrap)": np.round(th_mean, 6),
    "ส่วนเบี่ยงเบนมาตรฐาน": np.round(th_std, 6),
})

# สร้าง meta
meta = pd.DataFrame({
    "วิธีที่ดีที่สุด (6 คลาส)": [best_method6],
    "โหมด Threshold": [("policy-lock + target-quantile" if (POLICY_LOCK_QUANTILES and USE_TARGET_PREVALENCE)
                    else ("policy-lock q20/40/60/80" if POLICY_LOCK_QUANTILES else f"tuned (step={COARSE_STEP})"))],
    "เกณฑ์ Thresholds": [str(THRESHOLDS)],
    "ตัวชี้วัดหลัก": [("Macro-F1" if OPTIMIZE_FOR == "f1" else "Expected Cost (ยิ่งน้อยยิ่งดี)")],
    "คะแนน CV Macro-F1 (6 คลาส)": [float(cv_df6.iloc[0, 0])],
    "คะแนน Nested-CV หลัก": [nested_cv_score if nested_cv_score is not None else np.nan],
    "จำนวนแถวทั้งหมด": [len(df)],
    "จำนวนแถวที่มีความเสียหาย": [int(mask_has_damage.sum())],
    "จำนวนแถวที่ไม่มีความเสียหาย": [int((~mask_has_damage).sum())],
    "ค่า Random Seed": [RANDOM_SEED],
    "บันทึก": [
        ("อ้างอิง Nested-CV เป็นตัวตัดสินใจหลัก; "
         + ("คุมสัดส่วนด้วย target-quantile" if USE_TARGET_PREVALENCE else "ใช้ควอนไทล์ 20/40/60/80")
         + f"; min_gap={MIN_GAP:.02f}; optimize_for={OPTIMIZE_FOR}")
    ],
})

# =============================
# Export → Excel (ชื่อไทย)
# =============================
df_out = df.rename(columns={
    "TOTAL_DMG_FINAL": "คะแนนรวมความเสียหาย",
    "TOTAL_DMG_FINAL_NORM": "คะแนนรวมความเสียหาย(ปรับสเกล)",
    "DMG_LEVEL_FINAL": "ระดับความเสียหาย(สุดท้าย)",
    "DMG_CODE": "รหัสความเสียหาย",
})

with pd.ExcelWriter(out_path) as writer:
    df_out.to_excel(writer, sheet_name="ข้อมูล", index=False)
    weights_table.to_excel(writer, sheet_name="น้ำหนักสุดท้าย", index=False)
    cv_df6.to_excel(writer, sheet_name="คะแนนCV-6คลาส")
    norm_params.to_excel(writer, sheet_name="พารามิเตอร์การทำมาตรฐาน", index=False)
    meta.to_excel(writer, sheet_name="เมตาดาต้า", index=False)
    report_df.to_excel(writer, sheet_name="รายงานการจำแนก")
    cm_df.to_excel(writer, sheet_name="เมทริกซ์สับสน")
    stability_df.to_excel(writer, sheet_name="Bootstrap-ชั้น", index=False)
    th_stability_df.to_excel(writer, sheet_name="Bootstrap-Thresholds", index=False)



print(f"\n📁 เขียนไฟล์หลายชีตเรียบร้อย (ชื่อชีต/คอลัมน์ภาษาไทย): {out_path}")


✅ มีความเสียหาย: 2248 แถว | ไม่มีความเสียหาย: 4317 แถว
📊 คะแนน CV Macro-F1 (6 คลาส, group-aware):
              Macro-F1 (6 คลาส)
XGB                    0.946563
RF                     0.863923
Ridge                  0.830022
MI                     0.670828
ผู้เชี่ยวชาญ           0.614804

✅ วิธีที่ดีที่สุด (6 คลาส): XGB
🔒 เกณฑ์(Thresholds) policy-lock: [0.20724839898564215, 0.3258889165172987, 0.44001517176695937, 0.6472635707526015] | CV Macro-F1(6 คลาส) = 0.9706
🧪 Nested-CV Macro-F1 (threshold mode: policy-lock + target-quantile, min_gap=0.08): 0.6952
🔁 Bootstrap thresholds mean: [0.2055, 0.3251, 0.4335, 0.6487] | std: [0.0022, 0.0017, 0.0173, 0.0209]

การกระจายระดับความเสียหาย (หลังตั้งเกณฑ์):
DMG_LEVEL_FINAL
ไม่มีความเสียหาย    4317
น้อยมาก             1029
น้อย                 663
ปานกลาง              281
รุนแรง               179
วิกฤต                 96
Name: count, dtype: int64

📁 เขียนไฟล์หลายชีตเรียบร้อย (ชื่อชีต/คอลัมน์ภาษาไทย): C:\Users\piriy\Desktop\dataset_with_final_weig